In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(r"El_Paso_NOAA_data\NOAA_El_Paso_1998-2000.csv")

# Need to have separate dataframes for every distinct year of NOAA data and write each year to csv

In [ ]:
set_of_stations = set()

for row in df.iterrows():
#     print(row[1]['STATION'])
    temp_tuple = (row[1]['STATION'], row[1]['LATITUDE'], row[1]['LONGITUDE'])
    set_of_stations.add(temp_tuple)

In [ ]:
len(set_of_stations)

In [ ]:
df = pd.read_excel(r"El_Paso_NOAA_data\NOAA_El_Paso_1998-2019.xlsx")

In [ ]:
df1 = df['DATE'].tolist()
len(df1)

In [ ]:
count = 0

for element in df1:
    if element.year == 2001:
        count += 1
        
print(count)

In [ ]:
count = 0

for year in range(1998, 2020):
    
    temp_list = []
    temp_set = set()
    
    for i, row in df.iterrows():
        print(i)
        
        if year == row['DATE'].year:
            temp_tuple = (row['STATION'], row['LATITUDE'], row['LONGITUDE'])
            temp_set.add(temp_tuple)
    
    my_list = list(temp_set)

    with open('weather_stations_{}.txt'.format(year), 'w') as fp:
        fp.write('\n'.join('{},{},{}'.format(x[0],x[1], x[2]) for x in my_list))

In [ ]:
len(temp_set)

In [ ]:
my_list = list(temp_set)

with open('test.txt', 'w') as fp:
    fp.write('\n'.join('{},{},{}'.format(x[0],x[1], x[2]) for x in my_list))

In [ ]:
df_merged = pd.concat(temp_list)

In [ ]:
df_merged.to_csv("test.csv")

In [ ]:
tempDataList = []

with open('El_Paso_Weather_Stations/weather_stations_2019.txt'.format(year), 'r') as f:
    for line in f:
#         print(line.split(','))
        tempDataList.append({'ID': line.split(',')[0], 'lat': line.split(',')[1], 'lon': line.split(',')[2].replace('\n', '')})

# Based on the Data_Exploration_Integration files, remove all rows that are not bridges located in El Paso County

# Using geolocator to search every coordinate pair to compare the county to see if it belongs to El Paso County or not would take approx. 15 - 18 hours so I approximated the bounding box of El Paso County to compare the numerical coordinates rather than use a geolocator which takes time

# I used: https://boundingbox.klokantech.com/ to form an approximate bounding box for El Paso County which returned min latitude: 254.929456, min longitude: 38.519019, max latitude: 255.948379, max longitude: 39.130161

# First run is for Data_Exploration_Integration_1.csv

In [ ]:
import pandas as pd

for idx in range(1, 10):
    NBI_df = pd.read_csv('Data_Exploration_Integration_{}.csv'.format(idx))

    NBI_df = NBI_df.dropna(subset=['LAT_016'])
    NBI_df = NBI_df.dropna(subset=['LONG_017'])

    NBI_df['LAT_016'] = NBI_df['LAT_016'].astype(int)
    NBI_df['LONG_017'] = NBI_df['LONG_017'].astype(int)

    def dms2dd(s):

        minutes = s[-6:-4]
        seconds = s[-4:]
        seconds_decimal = s[-4:-2] + "." +s[-2:]

        temp_dms = minutes
        temp_dms = temp_dms + seconds

        if temp_dms in s:
            degrees = s.replace(temp_dms,'')

        dd = float(degrees) + float(minutes)/60 + float(seconds_decimal)/(60*60);

        return dd

    decimal_lats = []
    decimal_longs = []

    for index, row in NBI_df.iterrows():
        str_lat = str(row["LAT_016"])
        str_long = str(row["LONG_017"])

        if len(str_lat) < 8 or len(str_long) < 8:
            NBI_df = NBI_df.drop([index])
            continue

        decimal_lat = dms2dd(str_lat)
        decimal_long = dms2dd(str_long)

        decimal_lats.append(decimal_lat)
        decimal_longs.append(decimal_long)

    decimal_lats
    decimal_longs

    decimal_longs = [360 - x for x in decimal_longs]

    df_new = pd.DataFrame()
    df_new['decimal_lats'] = decimal_lats
    df_new['decimal_longs'] = decimal_longs

    df_new

    NBI_df['LAT_016'] = decimal_lats
    NBI_df['LONG_017'] = decimal_longs

    count = 0

    for index, row in NBI_df.iterrows():
        count += 1
        print(count)

        try:
            #min latitude: 254.929456, min longitude: 38.519019, max latitude: 255.948379, max longitude: 39.130161
            if 38.519019 > row['LAT_016'] or 39.130161 < row['LAT_016'] or 254.929456 > row['LONG_017'] or 255.948379 < row['LONG_017']:
                NBI_df = NBI_df.drop([index])
        except:
            NBI_df = NBI_df.drop([index])
            continue

    NBI_df.to_csv('Data_Exploration_Integration_El_Paso_{}.csv'.format(idx))